In [41]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [42]:
house_dict = []

for page in range(15, 21):
#     Agency Listing
#     linked = 'https://www.zillow.com/ca/townhomes/' + str(page) + '_p/?searchQueryState=%7B%22mapBounds%22%3A%7B%22west%22%3A-132.16065190625%2C%22east%22%3A-106.45264409375%2C%22south%22%3A29.712790718202044%2C%22north%22%3A44.40704031881662%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A9%2C%22regionType%22%3A2%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22sf%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A6%2C%22category%22%3A%22cat1%22%2C%22pagination%22%3A%7B%22currentPage%22%3A' + str(page) + '%7D%7D'
    
#     Other Listing
    linked = 'https://www.zillow.com/ca/townhomes/' + str(page) + '_p/?searchQueryState=%7B%22mapBounds%22%3A%7B%22west%22%3A-132.16065190625%2C%22east%22%3A-106.45264409375%2C%22south%22%3A29.712790718202044%2C%22north%22%3A44.40704031881662%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A9%2C%22regionType%22%3A2%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%2C%22sf%22%3A%7B%22value%22%3Afalse%7D%2C%22mf%22%3A%7B%22value%22%3Afalse%7D%2C%22con%22%3A%7B%22value%22%3Afalse%7D%2C%22land%22%3A%7B%22value%22%3Afalse%7D%2C%22apa%22%3A%7B%22value%22%3Afalse%7D%2C%22manu%22%3A%7B%22value%22%3Afalse%7D%2C%22apco%22%3A%7B%22value%22%3Afalse%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A6%2C%22category%22%3A%22cat2%22%2C%22pagination%22%3A%7B%22currentPage%22%3A' + str(page) + '%7D%7D'
    user_agent = 'Mozilla/5.0'
    headers = {'User-Agent': user_agent}
    response = requests.get(linked, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='list-card')
    print('page: ', page)
    
    # Loop through returned results
    for result in results:
        # Error handling
        try:
            # Identify and return housing weblink
            house_website = str(result.a.get('href'))
            print(house_website)
            # Identify and return house data
#             listing_type = result.find('div', class_="list-card-footer").text;
            price = result.find('div', class_="list-card-price").text
#             print(price)
            full_address = result.find('address', class_="list-card-addr").text
            street = result.find('address', class_="list-card-addr").text.split(",")[0]
            try:
                city = result.find('address', class_="list-card-addr").text.split(",")[1].replace(" ", "")
            except IndexError:
                city = 'null'
            try:
                state = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[1]
            except IndexError:
                state = 'null'
            try:
                zip_code = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[2]
            except IndexError:
                zip_code = 'null'
            bed = result.find('ul', class_="list-card-details").find('li').text.split(" ")[0]
            bath = result.find('ul', class_="list-card-details").select('li')[1].text.split(" ")[0]
            house_square_feet = result.find('ul', class_="list-card-details").select('li')[2].text.split(" ")[0]
            # Loop inside each house website and get more house data
            loop_response = requests.get(house_website, headers=headers)
            loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
            loop_result1 = loop_soup.find('ul', class_="ds-home-fact-list")
            loop_result2 = loop_soup.findAll('li', class_="ds-home-fact-list-item")

            lot = 'null';
            house_type = 'null';
            year_build = 'null';
            parking = 'null';
            hoa = 'null';
            lot_conversion_to_sqft = 'null';
            zillow_price_per_sqft = 'null';
            
            lot_search = loop_soup.findAll('div', class_="sc-pJipy hgzjTs")
            for content in lot_search:
#                 print(content.text)
                if "LotLot size" in content.text.split(":")[0]:
                    lot = content.text.split(": ")[1];
                    lot = lot.replace('Lot features', "")
            
            for content in loop_result2:
                if "Lot" in content.text.split(":")[0]:
                    lot = content.text.split(":")[1];
                if "Type" in content.text.split(":")[0]:
                    house_type = content.text.split(":")[1];
                if "Year built" in content.text.split(":")[0]:
                    year_build = content.text.split(":")[1];
                if "Parking" in content.text.split(":")[0]:
                    parking = content.text.split(":")[1];
                if "HOA" in content.text.split(":")[0]:
                    hoa = content.text.split(":")[1];
                if "Price/sqft" in content.text.split(":")[0]:
                    zillow_price_per_sqft = content.text.split(":")[1];
                    
            try:
                if (str(lot.split(" ")[1]) == "sqft"):
                    lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""));
                elif ((str(lot.split(" ")[1]) == "acres") or (str(lot.split(" ")[1]) == "acre") or (str(lot.split(" ")[1]) == "Acres") or (str(lot.split(" ")[1]) == "Acre")):
                    lot_conversion_to_sqft = round(float(lot.split(" ")[0].replace(',', "")) * 43560, 2);
                elif (lot == "No Data"):
                    lot_conversion_to_sqft = 'null'
                else:
                    lot_conversion_to_sqft = 'null'
            except IndexError:
                lot_conversion_to_sqft = 'null'
                
            if (len(loop_soup.findAll('span', class_="Text-c11n-8-38-0__aiai24-0 cMFTfb")) < 3):
                zestimate = 'null'
                rent_zestimate = 'null'
            else:
                zestimate = loop_soup.findAll('span', class_="Text-c11n-8-38-0__aiai24-0 cMFTfb")[0].text
                rent_zestimate = loop_soup.findAll('span', class_="Text-c11n-8-38-0__aiai24-0 cMFTfb")[2].text
            if (zestimate[0] != '$'):
                zestimate = 'null';
            print('zestimate', zestimate)
            print('rent', rent_zestimate)

            # Store data to house object if all elements are available
#             if (house_website and listing_type and price and full_address and street and city and state and zip_code and bed and bath and house_square_feet and house_type and year_build):
            house_obj = {
#                 "Listing_type": listing_type,
                "Price": price,
                "Full_address": full_address,
                "Street": street,
                "City": city,
                "State": state,
                "Zip_code": zip_code,
                "Bed": bed,
                "Bath": bath,
                "Square_Feet": house_square_feet,
                "Lot": lot,
                "Lot_conversion_to_sqft": lot_conversion_to_sqft,
                "House_type": house_type,
                "Year_build": year_build,
                "Parking": parking,
                "HOA": hoa,
                "Zestimate": zestimate,
                "Rent_zestimate": rent_zestimate,
                "Zillow_price/sqft": zillow_price_per_sqft,
                "House_website": house_website
            }

            #Continue to add data into collection house_dict
            house_dict.append(house_obj)
            time.sleep(5)
            
        except AttributeError as e:
            continue

# print(house_dict)

page:  15
https://www.zillow.com/homedetails/205-W-Maple-St-APT-10-Glendale-CA-91204/66674851_zpid/
zestimate $584,200
rent $2,434/mo
https://www.zillow.com/homedetails/6258-Nita-Ave-Los-Angeles-CA-91367/19929523_zpid/
zestimate $603,600
rent $2,999/mo
https://www.zillow.com/homedetails/5269-Colodny-Dr-UNIT-8-Agoura-Hills-CA-91301/19888585_zpid/
zestimate $589,800
rent $3,266/mo
https://www.zillow.com/homedetails/22771-Walnut-Park-Ln-Santa-Clarita-CA-91321/250346233_zpid/
zestimate $637,800
rent $3,313/mo
https://www.zillow.com/homedetails/24327-Avenida-De-Los-Ninos-22-Laguna-Niguel-CA-92677/25754103_zpid/
zestimate $532,400
rent $2,460/mo
https://www.zillow.com/homedetails/418-Carmel-Creeper-Pl-Encinitas-CA-92024/16714009_zpid/
zestimate $850,341
rent $3,134/mo
https://www.zillow.com/homedetails/574-S-Melrose-St-Anaheim-CA-92805/137355612_zpid/
zestimate $664,500
rent $2,600/mo
https://www.zillow.com/homedetails/2435-Orange-Ave-APT-A1-Costa-Mesa-CA-92627/25477371_zpid/
zestimate $902,

In [43]:
house_data = pd.DataFrame(house_dict)
house_data

,Price,Full_address,Street,City,State,Zip_code,Bed,Bath,Square_Feet,Lot,Lot_conversion_to_sqft,House_type,Year_build,Parking,HOA,Zestimate,Rent_zestimate,Zillow_price/sqft,House_website
0,"$558,900","205 W Maple St APT 10, Glendale, CA 91204",205 W Maple St APT 10,Glendale,CA,91204,2,3,900,"8,689 sqft",8689,Townhouse,1990,2 Garage spaces,$200 monthly,"$584,200","$2,434/mo",$621,https://www.zillow.com/homedetails/205-W-Maple...
1,"$648,900","6258 Nita Ave, Los Angeles, CA 91367",6258 Nita Ave,LosAngeles,CA,91367,2,2,"1,416",2.91 Acres,126760,Townhouse,1973,2 Parking spaces,$340 monthly,"$603,600","$2,999/mo",$458,https://www.zillow.com/homedetails/6258-Nita-A...
2,"$579,900","5269 Colodny Dr UNIT 8, Agoura Hills, CA 91301",5269 Colodny Dr UNIT 8,AgouraHills,CA,91301,3,3,"1,215",1.86 Acres,81021.6,Townhouse,1979,2 Garage spaces,$345 monthly,"$589,800","$3,266/mo",$477,https://www.zillow.com/homedetails/5269-Colodn...
3,"$637,900","22771 Walnut Park Ln, Santa Clarita, CA 91321",22771 Walnut Park Ln,SantaClarita,CA,91321,4,3,"1,664",1.10 Acres,47916,Townhouse,2017,2 Parking spaces,$116 monthly,"$637,800","$3,313/mo",$383,https://www.zillow.com/homedetails/22771-Walnu...
4,"$511,900","24327 Avenida De Los Ninos #22, Laguna Niguel,...",24327 Avenida De Los Ninos #22,LagunaNiguel,CA,92677,2,3,"1,024",No Data,null,Townhouse,1984,1 Garage space,$475 monthly,"$532,400","$2,460/mo",$500,https://www.zillow.com/homedetails/24327-Aveni...
5,"$826,200","418 Carmel Creeper Pl, Encinitas, CA 92024",418 Carmel Creeper Pl,Encinitas,CA,92024,2,3,"1,088",1.42 Acres,61855.2,Townhouse,1999,1 Garage space,$231 monthly,"$850,341","$3,134/mo",$759,https://www.zillow.com/homedetails/418-Carmel-...
6,"$646,900","574 S Melrose St, Anaheim, CA 92805",574 S Melrose St,Anaheim,CA,92805,2,2,"1,230",No Data,null,Townhouse,2012,2 Garage spaces,$227 monthly,"$664,500","$2,600/mo",$526,https://www.zillow.com/homedetails/574-S-Melro...
7,"$869,900","2435 Orange Ave APT A1, Costa Mesa, CA 92627",2435 Orange Ave APT A1,CostaMesa,CA,92627,3,3,"1,342","1,873 sqft",1873,Townhouse,1984,2 Garage spaces,$301 monthly,"$902,626","$3,528/mo",$648,https://www.zillow.com/homedetails/2435-Orange...
8,"$569,900","2021 Haller St #1, San Diego, CA 92104",2021 Haller St #1,SanDiego,CA,92104,2,3,"1,302","1,171 sqft",1171,Townhouse,1986,Carport,$190 monthly,"$584,327","$3,204/mo",$438,https://www.zillow.com/homedetails/2021-Haller...
9,"$558,900","205 W Maple St APT 10, Glendale, CA 91204",205 W Maple St APT 10,Glendale,CA,91204,2,3,900,"8,689 sqft",8689,Townhouse,1990,2 Garage spaces,$200 monthly,"$584,200","$2,434/mo",$621,https://www.zillow.com/homedetails/205-W-Maple...


In [44]:
# Export file as a CSV, without the Pandas index, but with the header
# house_data.to_csv("Output/Zillow/Q2-2021_Townhouse_Data_AgentListing_page15-20.csv", index=False, header=True)
house_data.to_csv("Output/Zillow/Q2-2021_Townhouse_Data_OtherListing_page15-20.csv", index=False, header=True)